In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [1]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from functools import partial
import sklearn.datasets
import scipy.linalg
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

Autosave disabled



# Python y rendimiento 2

La clase anterior vimos como hacer *profiling* de nuestro código con el fin de diagnosticar las secciones menos eficientes

En este clase veremos como optimizar un código Python para mejorar su rendimiento


# Optimización

Se refiere a modificar una rutina para mejorar su eficiencia, es decir reducir sus

1. Tiempos de ejecución
1. Consumo de recursos 

La optimización debería concentrarse en los **cuellos de botella** que revelamos al hacer *profiling* de nuestro código


## Optimización en Python

Algunos consejos con respecto a la optimización de código en Python

### 1.  Evita optimizar prematuramente

Este consejo aplica para todos los lenguajes de programación

Optimizar toma tiempo. 
No dediques esfuerzo a optimizar si tu código aun está completo o funcional

- Optimizar una sección que no es tan relevante
- Introducir un bug al optimizar

> Premature optimization is the root of all evil - Donald Knuth 

https://effectiviology.com/premature-optimization/

https://en.wikipedia.org/wiki/Program_optimization#When_to_optimize

### 2. Haz buen uso de la sintaxis y funciones de Python

https://ipython-books.github.io/51-knowing-python-to-write-faster-code/

https://wiki.python.org/moin/PythonSpeed/PerformanceTips


- Cython
- Multiprocessing

https://pjryan126.github.io/multiprocessing-in-python/

https://github.com/jupyter/notebook/issues/1703

https://medium.com/@grvsinghal/speed-up-your-code-using-multiprocessing-in-python-36e4e703213e

https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac


http://people.duke.edu/~ccc14/sta-663-2017/10A_CodeOptimization.html